# cs542 Machine Learning

Collaborator: Yiteng Xu, Changlei Shi, ChenCheng Wang, Yi Li

# Pre-process data

In [2]:
import csv
import pandas as pd
import numpy as np
import json
from collections import defaultdict

def readJson(f):
    for l in open(f):
        yield eval(l)

In [19]:
items = []

for l in open("anonymous-msweb.data"):
    if l.startswith("A"):
        m,n,p,q,i = l.strip().split(',')
        q = q[1:-1]
        i = i[1:-1]
        tmp = [n, q, i]
        items.append(tmp)

items_df = pd.DataFrame(items)
items_df.to_csv("item.txt")

In [27]:
current = ""
item = ""
user_item = []
training_user_item_dict = {}

for l in open("anonymous-msweb.data"):
    if l.startswith("C"):
        m,n,p = l.strip().split(',')
        current = p
    if l.startswith("V"):
        a,b,c = l.strip().split(',')
        item = b
        tp = [current, item]
        user_item.append(tp)
        if current in training_user_item_dict:
            cur_list = training_user_item_dict[current]
            cur_list.append(item)
            training_user_item_dict[current] = cur_list
        else:
            my_list = [item]
            training_user_item_dict[current] = my_list
user_item_df = pd.DataFrame(user_item)
user_item_df.to_csv("user_item.txt")

json_str = json.dumps(training_user_item_dict)
data = json.loads(json_str)
with open('user_item_dict.json', 'w') as f:
    json.dump(data, f)
#print training_user_item_dict
'''
w = csv.writer(open("user_item_dict.csv", "w"))
for key, val in training_user_item_dict.items():
    w.writerow([key, val])
'''

'\nw = csv.writer(open("user_item_dict.csv", "w"))\nfor key, val in training_user_item_dict.items():\n    w.writerow([key, val])\n'

In [26]:
import json
items = []

for l in open("anonymous-msweb.test"):
    if l.startswith("A"):
        m,n,p,q,i = l.strip().split(',')
        q = q[1:-1]
        i = i[1:-1]
        tmp = [n, q, i]
        items.append(tmp)

items_df = pd.DataFrame(items)
items_df.to_csv("item_test.txt")

current = ""
item = ""
user_item = []
training_user_item_dict = {}
for l in open("anonymous-msweb.test"):
    if l.startswith("C"):
        m,n,p = l.strip().split(',')
        current = p
    if l.startswith("V"):
        a,b,c = l.strip().split(',')
        item = b
        tp = [current, item]
        user_item.append(tp)
        if current in training_user_item_dict:
            cur_list = training_user_item_dict[current]
            cur_list.append(item)
            training_user_item_dict[current] = cur_list
        else:
            my_list = [item]
            training_user_item_dict[current] = my_list
user_item_df = pd.DataFrame(user_item)
user_item_df.to_csv("user_item_test.txt")

#print training_user_item_dict
json_str = json.dumps(training_user_item_dict)
data = json.loads(json_str)
with open('user_item_dict_test.json', 'w') as f:
    json.dump(data, f)

'''
w = csv.writer(open("user_item_dict_test.csv", "w"))
for key, val in training_user_item_dict.items():
    w.writerow([key, val])
'''

In [23]:
import csv
dict = {}
for key, val in csv.reader(open("user_item_dict_test.csv")):
    print key
    val[1:-1].replace('\'', '')
    print str(val[1:-1]).split(",")
    break

14478
["'1136'", " '1046'", " '1018'"]


# Implementing Memory-based Algorithms

In [1]:
import csv
import pandas as pd
import numpy as np
import json
from collections import defaultdict

#load file
user_item_df = pd.read_csv("user_item.txt")
user_item_df.columns = ['index','user', 'item']
user_item_df.drop('index', axis=1, inplace=True)

#group by user and get the total vote of each user
tmp = user_item_df.groupby(['user']).count()
user_votes = []
for index, item in tmp.iterrows():
    c = int(item['item'])
    tp = [index, c]
    user_votes.append(tp)

#save results to another dataframe
user_votes_df = pd.DataFrame(user_votes)
user_votes_df.columns = ['user','count']

#print the mean of votes per user
print user_votes_df['count'].mean()
#print number of users in trainning set
print len(user_votes_df)


3.01592736388
32711


In [ ]:
import math
def prediction(user_a, item_j, compare_users_data):
    total = 0
    for user_i, _list in compare_users_data:
        if user_a is not user_i:
            v_ij = 0
            if item_j in _list:
                v_ij = 1
            w = getWeight(user_a, user_i, compare_users_data)
            total = total + w * (v_ij - 1)
    return 1 + k * total


In [9]:
import json
import math

def getWeight(a, i, compare_users_data):
    set1 = set(compare_users_data[a])
    set2 = set(compare_users_data[i])
    length1 = len(set1)
    length2 = len(set2)
    inter = set1.intersection(set2)
    total = 0
    for j in inter:
        den1 = math.sqrt(length1)
        den2 = math.sqrt(length2)
        total = total + (float(1) / den1) * (float(1) / den2)
    return total

'''
items_df = pd.read_csv("user_item_test.txt")
u_i_df = pd.read_csv("user_item.txt")

compare_users = []
for index, item in u_i_df.iterrows():
    compare_users.append(str(item[1]))

active_users = []
for index, item in items_df.iterrows():
    active_users.append(str(item[1]))
    
active_users = list(set(active_users))
compare_users = list(set(compare_users))

'''

with open('user_item_dict.json') as data_file:    
    compare_users_data = json.load(data_file)
    
#with open('user_item_dict_test.json') as data_file:    
#    active_users_data = json.load(data_file)

#test
print getWeight("13922","11541", compare_users_data)       

print compare_users_data["13922"]
print compare_users_data["11541"]

0.377964473009
[u'1008', u'1009', u'1177', u'1018']
[u'1008', u'1089', u'1017', u'1052', u'1040', u'1048', u'1018']


Code for setting the style of the notebook

In [1]:

from IPython.core.display import HTML
def css_styling():
    styles = open("../../theme/custom.css", "r").read()
    return HTML(styles)
css_styling()

IOError: [Errno 2] No such file or directory: '../../theme/custom.css'